In [35]:
from __future__ import print_function

import numpy as np
import scipy.misc
import scipy.io
import scipy as sc
import tensorflow as tf
from PIL import Image

from keras.layers import Input
from keras.applications import vgg19
from keras.optimizers import SGD
from keras.models import Model

import keras.backend as K

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
%matplotlib inline

In [2]:
class CONFIG:
    IMAGE_WIDTH = 400
    IMAGE_HEIGHT = 300
    COLOR_CHANNELS = 3
    CONTENT_WEIGHT = 5
    STYLE_WEIGHT = 100
    TOTAL_VARIATION_WEIGHT = 1.
    NOISE_RATIO = .6
    MEANS = np.array([123.68, 116.779, 103.939]).reshape((1,1,1,3))

In [3]:
STYLE_LAYERS = [
    ('block1_conv1', .5),
#     ('block1_conv2', .2),
    ('block2_conv1', .5),
#     ('block2_conv2', .2),
    ('block3_conv1', .5),
#     ('block3_conv2', 0.2),
#     ('block3_conv3', .2),
#     ('block3_conv4', 0.2),
    ('block4_conv1', .5),
#     ('block4_conv2', 0.2),
#     ('block4_conv3', 0.2),
#     ('block4_conv4', .2),
    ('block5_conv1', .5),
#     ('block5_conv2', 0.2),
#     ('block5_conv3', 0.2),
#     ('block5_conv4', .2)
]

In [4]:
def reshape_and_normalize_image(image_path):
    """
    Reshape and normalize the input image (content or style)
    """
    
    image = Image.open(image_path)
    image = image.resize((CONFIG.IMAGE_WIDTH, CONFIG.IMAGE_HEIGHT))
    
    image_array = np.asarray(image, dtype='float32')
    image = np.expand_dims(image_array, axis=0)
    
    # Substract the mean to match the expected input of VGG16
    image = image - CONFIG.MEANS
    
    return image

In [5]:
def generate_noise_image(content_image, noise_ratio = CONFIG.NOISE_RATIO):
    """
    Generates a noisy image by adding random noise to the content_image
    """
    
    # Generate a random noise_image
    noise_image = np.random.uniform(-20, 20, (1, CONFIG.IMAGE_HEIGHT, CONFIG.IMAGE_WIDTH,
                                              CONFIG.COLOR_CHANNELS)).astype('float32')
    
    # Set the input_image to be a weighted average of the content_image and a noise_image
    input_image = noise_image * noise_ratio + content_image * (1 - noise_ratio)
    
    return input_image

In [6]:
content_image_path = "./images/louvre_small.jpg"
content_image = reshape_and_normalize_image(content_image_path)

style_image_path = "./images/monet.jpg"
style_image = reshape_and_normalize_image(style_image_path)

generated_image = generate_noise_image(content_image)

In [7]:
def compute_content_loss(content, generated):
    m, n_H, n_W, n_C = generated.shape

    content = np.reshape(content, [-1])
    generated = np.reshape(generated, [-1])

    return np.sum(np.square(content - generated)) / (4*n_H*n_W*n_C)

In [8]:
def gram_matrix(x):
    gram = np.dot(np.transpose(x), x)
    return gram

def style_layer_loss(style, generated):
    m, n_H, n_W, n_C = generated.shape
    
    style = np.reshape(style, (n_H*n_W, n_C))
    generated = np.reshape(generated, (n_H*n_W, n_C))
    
    S = gram_matrix(style)
    G = gram_matrix(generated)

    channels = n_C
    size = n_H*n_W

    return np.sum(np.square(S - G)) / (4. * (channels ** 2) * (size ** 2))

In [9]:
sess = tf.Session()
K.set_session(sess)

In [10]:
input_shape = (CONFIG.IMAGE_HEIGHT, CONFIG.IMAGE_WIDTH, CONFIG.COLOR_CHANNELS)
model = vgg19.VGG19(weights=None, input_shape=input_shape, include_top=False)
model.load_weights("./vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5")

In [11]:
# get the symbolic outputs of each "key" layer (we gave them unique names).
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

In [12]:
# make list of all the layers defined in STYLE_LAYERS
layer_tensors = [outputs_dict[layer_name] for layer_name, _ in STYLE_LAYERS]
layer_tensors.append(model.input)

In [13]:
new_model = Model(inputs=model.input, outputs=layer_tensors)

In [14]:
def total_loss(alpha=20, beta=40):
    
    content_layers = new_model.predict(content_image)
    generated_layers = new_model.predict(generated_image)

    # Get content loss from output of block 4, layer 2
    content_features = content_layers[-2]
    generated_features = generated_layers[-2]
    content_loss = compute_content_loss(content_features, generated_features)
    
    # loss for style image
    style_loss = 0.
    style_layers = new_model.predict(style_image)
    
    i = len(STYLE_LAYERS)
    for _, coeff in STYLE_LAYERS:
        # Select the output tensor of the currently selected layer
        style_features = style_layers[-i]
        generated_features = generated_layers[-i]
        style_loss += coeff * style_layer_loss(style_features, generated_features)
        i -= 1
    
    # Get total loss using alpha and beta
    total_loss = (content_loss*alpha) + (style_loss*beta)
    
    return total_loss

In [15]:
J = total_loss()

In [16]:
new_model.predict(generated_image)[-2]

array([[[[  5.10645485e+01,   0.00000000e+00,   7.23399582e+01, ...,
            4.46864243e+01,   0.00000000e+00,   0.00000000e+00],
         [  2.00483532e+01,   0.00000000e+00,   0.00000000e+00, ...,
            0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
         [  3.05583630e+01,   0.00000000e+00,   0.00000000e+00, ...,
            0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
         ..., 
         [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
            0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
         [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
            0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
         [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
            0.00000000e+00,   0.00000000e+00,   2.44372582e+00]],

        [[  5.40156794e+00,   0.00000000e+00,   1.04163216e+02, ...,
            6.66441584e+00,   0.00000000e+00,   0.00000000e+00],
         [  0.00000000e+00,   0.00000000e+00,

In [17]:
sgd = SGD(lr=0.5, decay=1e-6, momentum=0.99, nesterov=True)
new_model.compile(loss=total_loss, optimizer=sgd)

In [18]:
new_model.predict(generated_image)[-2]

array([[[[  5.10645485e+01,   0.00000000e+00,   7.23399582e+01, ...,
            4.46864243e+01,   0.00000000e+00,   0.00000000e+00],
         [  2.00483532e+01,   0.00000000e+00,   0.00000000e+00, ...,
            0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
         [  3.05583630e+01,   0.00000000e+00,   0.00000000e+00, ...,
            0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
         ..., 
         [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
            0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
         [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
            0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
         [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
            0.00000000e+00,   0.00000000e+00,   2.44372582e+00]],

        [[  5.40156794e+00,   0.00000000e+00,   1.04163216e+02, ...,
            6.66441584e+00,   0.00000000e+00,   0.00000000e+00],
         [  0.00000000e+00,   0.00000000e+00,

In [18]:
def save_image(path, image):
    
    # Un-normalize the image so that it looks good
    image = image + CONFIG.MEANS
    
    # Clip and Save the image
    image = np.clip(image[0], 0, 255).astype('uint8')
    scipy.misc.imsave(path, image)

In [19]:
# save last generated image
save_image('output/generated_image_keras.jpg', generated_image)

/Users/apple/anaconda3/envs/coreml/lib/python2.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  


In [20]:
new_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 300, 400, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 300, 400, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 300, 400, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 150, 200, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 150, 200, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 150, 200, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 75, 100, 128)      0         
__________